In [8]:
import pandas as pd
import gspread

from typing import Any

json_file_path = "./data/drive-sheet-01.json"

In [9]:
def googleSpreadData(url: str) -> Any:
    if url is None:
        return None
    
    gc = gspread.service_account(filename=json_file_path)
    # url = url

    ## 4. PythonSheet url을 바탕으로 스프레드 시트를 열고,
    doc = gc.open_by_url(url)

    workDf = pd.DataFrame()
    # print(doc.worksheets()[0].title)
    for sheet in doc.worksheets():
        print(sheet.title)
        data = doc.worksheet(sheet.title).get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])
        df['name'] = sheet.title
        workDf = pd.concat([workDf, df], ignore_index= True)
    # print(workDf)
    
    return workDf

In [10]:
from langchain.tools import tool

@tool
def searchSheetData(query:str) -> str:
    """Search for things done by person's name"""
    
    url = 'https://docs.google.com/spreadsheets/d/1rW-nTz7P69BHGCtp0amownjnuc2OJmFzJrQFoAfeKnc'
    dataDf = googleSpreadData(url)
    print(dataDf.head())
    return None

In [11]:
searchSheetData.invoke({"query": ""})

퍼머넌트
    Code      이름     거리    상승고도   제한시간    출발점    도착점           코스설계자  name
0  PT-02    푸른마음  202km  2,583m  13:28     왜관     왜관  Russell Morris  퍼머넌트
1  PT-03     강화도  210km  1,356m  14:00  서울 반포  서울 반포       Jason Ham  퍼머넌트
2  PT-04    모래시계  200km  2,191m  13:20  서울 반포  서울 반포       Jason Ham  퍼머넌트
3  PT-05  굿모닝 남해  202km  2,163m  13:28  부산 사상  부산 사상    Jan Boonstra  퍼머넌트
4  PT-06   인천-대전  214km  2,516m  14:16     인천     대전       Jason Ham  퍼머넌트


### CSV-Excel-Agent 사용해보기

In [22]:
from langchain_experimental.tools import PythonAstREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonAstREPLTool()

url = 'https://docs.google.com/spreadsheets/d/1rW-nTz7P69BHGCtp0amownjnuc2OJmFzJrQFoAfeKnc'
df = googleSpreadData(url)
python_tool.locals["df"] = df


# 도구 호출 시 실행되는 콜백 함수입니다.
def tool_callback(tool) -> None:
    print(f"<<<<<<< Code >>>>>>")
    if tool_name := tool.get("tool"):  # 도구에 입력된 값이 있다면
        if tool_name == "python_repl_ast":
            tool_input = tool.get("tool_input")
            for k, v in tool_input.items():
                if k == "query":
                    print(f"query : {v}")  # Query 를 출력합니다.
                    result = python_tool.invoke({"query": v})
                    print(result)
    print(f"<<<<<<< Code >>>>>>")


# 관찰 결과를 출력하는 콜백 함수입니다.
def observation_callback(observation) -> None:
    print(f"<<<<<<< Message >>>>>>")
    if "observation" in observation:
        print(observation["observation"])
    print(f"<<<<<<< Message >>>>>>")


# 최종 결과를 출력하는 콜백 함수입니다.
def result_callback(result: str) -> None:
    print(f"<<<<<<< 최종 답변 >>>>>>")
    print(result)
    print(f"<<<<<<< 최종 답변 >>>>>>")

퍼머넌트


In [23]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import AgentStreamParser, AgentCallbacks

agent = create_pandas_dataframe_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    df,
    verbose=False,
    agent_type="tool-calling",
    allow_dangerous_code=True,
    prefix="You are a professional data analyst and expert in Pandas. "
    "You must use Pandas DataFrame(`df`) to answer user's request. "
    "\n\n[IMPORTANT] DO NOT create or overwrite the `df` variable in your code. \n\n"
    "If you are willing to generate visualization code, please use `plt.show()` at the end of your code. "
    "I prefer seaborn code for visualization, but you can use matplotlib as well.",
)

parser_callback = AgentCallbacks(tool_callback, observation_callback, result_callback)
stream_parser = AgentStreamParser(parser_callback)

In [24]:
def ask(query):
    # 질의에 대한 답변을 출력합니다.
    response = agent.stream({"input": query})

    for step in response:
        stream_parser.process_agent_steps(step)

In [17]:
ask("몇 개의 행이 있어?")

<<<<<<< Code >>>>>>
len(df)
164
<<<<<<< Code >>>>>>
<<<<<<< Message >>>>>>
164
<<<<<<< Message >>>>>>
<<<<<<< 최종 답변 >>>>>>
데이터프레임에는 총 164개의 행이 있습니다.
<<<<<<< 최종 답변 >>>>>>


In [19]:
ask("서울 여의도에서 시작하는 코스 이름들과 가장 긴 거리의 코스가 어디이고 몇 km인지 알려줘")

<<<<<<< Code >>>>>>
df[df['출발점'] == '서울 여의도']
     Code        이름     거리    상승고도   제한시간     출발점     도착점 코스설계자  name
20  PT-26  청평호와 호명산  257.0  3,096m  17:08  서울 여의도  서울 여의도   장남식  퍼머넌트
21  PT-28       교동도  204.0  1,375m  13:36  서울 여의도  서울 여의도   장남식  퍼머넌트
<<<<<<< Code >>>>>>
<<<<<<< Message >>>>>>
     Code        이름     거리    상승고도   제한시간     출발점     도착점 코스설계자  name
20  PT-26  청평호와 호명산  257.0  3,096m  17:08  서울 여의도  서울 여의도   장남식  퍼머넌트
21  PT-28       교동도  204.0  1,375m  13:36  서울 여의도  서울 여의도   장남식  퍼머넌트
<<<<<<< Message >>>>>>
<<<<<<< Code >>>>>>
longest_course = df.loc[df['거리'].str.replace('km', '').astype(float).idxmax()]
longest_course_name = longest_course['이름']
longest_course_distance = longest_course['거리']
longest_course_name, longest_course_distance
AttributeError: Can only use .str accessor with string values!
<<<<<<< Code >>>>>>
<<<<<<< Message >>>>>>
AttributeError: Can only use .str accessor with string values!
<<<<<<< Message >>>>>>
<<<<<<< Code >>>>>>
df['거리'] = df['거리'].st

In [20]:
ask("거리가 가장 짧은 코스는 어디서 부터 어디까지 인지, 그리고 그 거리와 코스이름, 코스 설계자를 알려줘")

<<<<<<< Code >>>>>>
shortest_course = df.loc[df['거리'].str.replace('km', '').astype(float).idxmin()]
shortest_course_info = shortest_course[['출발점', '도착점', '거리', '이름', '코스설계자']]
shortest_course_info.to_dict()
AttributeError: Can only use .str accessor with string values!
<<<<<<< Code >>>>>>
<<<<<<< Message >>>>>>
AttributeError: Can only use .str accessor with string values!
<<<<<<< Message >>>>>>
<<<<<<< Code >>>>>>
df['거리'] = df['거리'].str.replace('km', '').astype(float)
shortest_course = df.loc[df['거리'].idxmin()]
shortest_course_info = shortest_course[['출발점', '도착점', '거리', '이름', '코스설계자']]
shortest_course_info.to_dict()
AttributeError: Can only use .str accessor with string values!
<<<<<<< Code >>>>>>
<<<<<<< Message >>>>>>
AttributeError: Can only use .str accessor with string values!
<<<<<<< Message >>>>>>
<<<<<<< Code >>>>>>
df['거리'] = df['거리'].astype(str).str.replace('km', '').astype(float)
shortest_course = df.loc[df['거리'].idxmin()]
shortest_course_info = shortest_course[['출발점', '도착

In [25]:
ask("거리가 가장 짧은 코스들은 몇 km인지와 코스 이름들, 시작지점 종료지점을 알려줘.")

<<<<<<< Code >>>>>>
query : min_distance = df['거리'].apply(lambda x: int(x.replace('km', ''))).min()
min_distance_courses = df[df['거리'].apply(lambda x: int(x.replace('km', ''))) == min_distance]
min_distance, min_distance_courses[['이름', '출발점', '도착점']]
(200,             이름    출발점    도착점
2         모래시계  서울 반포  서울 반포
18   아름다운 길 함양     함양     함양
33     충주 어드밴처  서울 잠실     충주
43         경춘선  서울 당산     춘천
49       스카이시티     인천     인천
87     밀양 어드벤처     밀양     밀양
137      파주 뱅뱅  서울 마포  서울 마포
138        사선대     전주     전주
142      한강 투어  서울 반포  서울 반포
161  제주 긴꼬리벵에돔    제주시    제주시)
<<<<<<< Code >>>>>>
<<<<<<< Message >>>>>>
(200,             이름    출발점    도착점
2         모래시계  서울 반포  서울 반포
18   아름다운 길 함양     함양     함양
33     충주 어드밴처  서울 잠실     충주
43         경춘선  서울 당산     춘천
49       스카이시티     인천     인천
87     밀양 어드벤처     밀양     밀양
137      파주 뱅뱅  서울 마포  서울 마포
138        사선대     전주     전주
142      한강 투어  서울 반포  서울 반포
161  제주 긴꼬리벵에돔    제주시    제주시)
<<<<<<< Message >>>>>>
<<<<<<< 최종 답변 >>>>>>
거리가 가장 짧은 